# Classify patches from D&F images.
Analzye tumor B13. Mostly blue but more geen than A3. 


In [1]:
import time
import os
import glob
import cv2 # OpenCV-Python
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import numpy as np
import json
DIR_IMAGES_RAW = "/home/jrm/Martinez/images/raw/"
DIR_IMAGES_OUT = "/home/jrm/Martinez/images/temp/"
PATTERN_IMAGES_RAW = "*.DF1.*.tif"
PATCH_SIZE=224  # matches VGG
IMAGE_FILENAME='B13.DF1.115.tif'

In [2]:
def get_image_names(path,pattern):
    #RAW_IMAGE_NAMES = os.listdir(DIR_IMAGES_RAW)
    paths = glob.glob(path+pattern)
    names = [os.path.basename(x) for x in paths]
    return names
FILENAMES_IMAGES_RAW = get_image_names(DIR_IMAGES_RAW,PATTERN_IMAGES_RAW)
#FILENAMES_IMAGES_RAW

In [3]:
def load_pixel_array(path,filename,verbose=False):
    im = Image.open(path+filename)
    ima = np.array(im)   # convert to numpy
    if verbose:
        print(filename, ima.size, ima.shape)
    return ima
pixel_array = load_pixel_array(DIR_IMAGES_RAW,IMAGE_FILENAME,True)

B13.DF1.115.tif 479598264 (12513, 12776, 3)


In [4]:
avg = np.average(pixel_array,axis=(0,1))
std = np.std(pixel_array,axis=(0,1))
print("Average",avg,"STD",std)
print("Example pixel:",pixel_array[0,0])

Average [ 0.         17.85214838 35.88764711] STD [ 0.         14.50691847 48.9509621 ]
Example pixel: [ 0 13  3]


In [5]:
PATCH_THRESHOLD=PATCH_SIZE*PATCH_SIZE/2
IM_WIDTH = pixel_array.shape[0]
IM_HEIGHT = pixel_array.shape[1]

In [6]:
# TO DO: Create an iterator class and dispense with the globals.
W=0
H=0
def get_next_patch(pixels): # assume square
    global W,H
    if W+PATCH_SIZE>IM_WIDTH:
        H += PATCH_SIZE
        W = 0
    if H+PATCH_SIZE>IM_HEIGHT:
        patch = None
    if W+PATCH_SIZE<=IM_WIDTH and H+PATCH_SIZE<=IM_HEIGHT:
        patch = pixels[W:W+PATCH_SIZE, H:H+PATCH_SIZE]
        W += PATCH_SIZE
    return patch

In [7]:
def pixel_to_heatmap(green,blue):   # TO DO: this is slow, need a hash function
    bins=[10,20,30,40,50,60,70,80,90,256]
    gbin=None
    bbin=None
    for bin in range(0,10):
        if gbin is None and green<=bins[bin]:
            gbin=bin
        if bbin is None and blue<=bins[bin]:
            bbin=bin
        if gbin is not None and bbin is not None:
            return gbin,bbin
    return gbin,bbin
def accumulate_pixels(imary,verbose=False):  # TO DO: nested for loop is very slow
    heatmap=np.zeros( (10,10), dtype=np.int32)
    if verbose:
        print("accumulate",imary.shape)
    nrows,ncols,nchannel=imary.shape
    for row in range(0,nrows):
        for col in range(0,ncols):
            pixel = imary[row,col]
            #red = pixel[0]
            green = pixel[1]
            blue = pixel[2]
            gbin,bbin = pixel_to_heatmap(green,blue)
            heatmap[gbin,bbin] += 1
    return heatmap

In [8]:
def save_patch (patch,name):
    pic = Image.fromarray(patch)
    pic.save(DIR_IMAGES_OUT+name)  
def show_heatmap (patch):
    hm = accumulate_pixels(patch)
    print(hm,name)        

In [11]:
W=0
H=0
BACKGROUND_MAX=np.array( [20,20,20] )
GREEN_MIN=np.array( [0,40,0] )
BLUE_MIN=np.array( [0,0,40] )
num_black_patches = 0
num_green_patches = 0
num_blue_patches = 0
num_patches = 0

if True:
    patch = get_next_patch(pixel_array)
    while patch is not None:
        num_patches += 1
        num_background_pixels = np.sum(np.all(patch<BACKGROUND_MAX,axis=2))
        if num_background_pixels>=PATCH_THRESHOLD:
            num_black_patches += 1
            if num_black_patches < 10:
                    name = "BLACK{}.tif".format(num_black_patches)
                    save_patch(patch,name)
                    show_heatmap(patch)
        else:
            num_green_pixels = np.sum(np.all(patch>=GREEN_MIN,axis=2))
            if num_green_pixels>=PATCH_THRESHOLD:
                num_green_patches += 1
                if num_green_patches < 10:
                        name = "GREEN{}.tif".format(num_green_patches)
                        save_patch(patch,name)
                        show_heatmap(patch)
            else:
                num_blue_pixels = np.sum(np.all(patch>=BLUE_MIN,axis=2))
                if num_blue_pixels>=PATCH_THRESHOLD:
                    num_blue_patches += 1
                    if num_blue_patches < 10:  # in [250,256,260]:
                        name = "BLUE{}.tif".format(num_blue_patches)
                        save_patch(patch,name)
                        show_heatmap(patch)
                    
        patch = get_next_patch(pixel_array)
    print("back=",num_back_patches,"green=",num_green_patches,"blue=",num_blue_patches,"total=",num_patches)
    print("back=",num_back_patches/num_patches,"green=",num_green_patches/num_patches,"blue=",num_blue_patches/num_patches)

[[ 9330     0     0     0     0     0     0     0     0     0]
 [40838     0     0     0     0     0     0     0     0     0]
 [    6     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    1     0     0     0     0     0     0     0     0     0]
 [    1     0     0     0     0     0     0     0     0     0]] BLACK1.tif
[[35610     1     0     0     0     0     0     0     0     0]
 [14563     0     0     0     0     0     0     0     0     0]
 [    1     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0

[[   0    1    0    2    1    0    0    0    0    0]
 [   0  376 3532 2846 2027 1279  588  468  348  561]
 [   0    0  451 2525 1139  491  772 1059  914 9470]
 [   0    0  259  886  840  742  519  532  635 8715]
 [   0    0   34  121  124  197  351  363  338 5445]
 [   0    0    3   31    4   60  131   59   70  766]
 [   0    0    0    1    0    2   14    3    1   79]
 [   0    0    0    0    0    0    0    0    0    1]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]] BLUE6.tif
[[   0    0    0    0    0    0    0    0    0    0]
 [   0    0 1109 3502 1147  320  131   96   47   53]
 [   0    0 1118 5356 4794 1413 1016  772  691 6683]
 [   0    0   12  710 1085  737  740  700  642 8390]
 [   0    0    1  243  251  488  315  256  235 4668]
 [   0    0    0  150   74  101  131  138  128 1298]
 [   0    0    0    4   44   38   27   29   25  261]
 [   0    0    0    0    0    0    3    0    1    3]
 [   0    0    0    0    0    0    